# Fake News Detection - Complete Notebook

In [1]:
# Install dependencies (only required in Colab)
!pip install -U nltk spacy gensim wordcloud
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of thinc to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 94.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 100.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.3
    Uninstalling scipy-1.15.3:
      Successfully uninstalled scipy-1.15.3
  Att

Traceback (most recent call last):
  File "<frozen runpy>", line 189, in _run_module_as_main
  File "<frozen runpy>", line 148, in _get_module_details
  File "<frozen runpy>", line 112, in _get_module_details
  File "/usr/local/lib/python3.11/dist-packages/spacy/__init__.py", line 6, in <module>
    from .errors import setup_default_warnings
  File "/usr/local/lib/python3.11/dist-packages/spacy/errors.py", line 3, in <module>
    from .compat import Literal
  File "/usr/local/lib/python3.11/dist-packages/spacy/compat.py", line 4, in <module>
    from thinc.util import copy_array
  File "/usr/local/lib/python3.11/dist-packages/thinc/__init__.py", line 5, in <module>
    from .config import registry
  File "/usr/local/lib/python3.11/dist-packages/thinc/config.py", line 5, in <module>
    from .types import Decorator
  File "/usr/local/lib/python3.11/dist-packages/thinc/types.py", line 25, in <module>
    from .compat import cupy, has_cupy
  File "/usr/local/lib/python3.11/dist-packages/t

In [1]:
import pandas as pd
import numpy as np
import nltk
import spacy
import string
import re
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, roc_auc_score
from gensim.models import Word2Vec
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nlp = spacy.load('en_core_web_sm')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


## 1. Data Preparation
### 1.0 Data Understanding

In [3]:
# Load True and Fake datasets
true_df = pd.read_csv('/content/True.csv')
fake_df = pd.read_csv('/content/True.csv')

In [4]:
# Inspect the DataFrame with True News to understand the given data
true_df.head()

title  \
0  As U.S. budget fight looms, Republicans flip t...   
1  U.S. military to accept transgender recruits o...   
2  Senior U.S. Republican senator: 'Let Mr. Muell...   
3  FBI Russia probe helped by Australian diplomat...   
4  Trump wants Postal Service to charge 'much mor...   

                                                text                date  
0  WASHINGTON (Reuters) - The head of a conservat...  December 31, 2017   
1  WASHINGTON (Reuters) - Transgender people will...  December 29, 2017   
2  WASHINGTON (Reuters) - The special counsel inv...  December 31, 2017   
3  WASHINGTON (Reuters) - Trump campaign adviser ...  December 30, 2017   
4  SEATTLE/WASHINGTON (Reuters) - President Donal...  December 29, 2017

In [5]:
# Inspect the DataFrame with Fake News to understand the given data
fake_df.head()

title  \
0  As U.S. budget fight looms, Republicans flip t...   
1  U.S. military to accept transgender recruits o...   
2  Senior U.S. Republican senator: 'Let Mr. Muell...   
3  FBI Russia probe helped by Australian diplomat...   
4  Trump wants Postal Service to charge 'much mor...   

                                                text                date  
0  WASHINGTON (Reuters) - The head of a conservat...  December 31, 2017   
1  WASHINGTON (Reuters) - Transgender people will...  December 29, 2017   
2  WASHINGTON (Reuters) - The special counsel inv...  December 31, 2017   
3  WASHINGTON (Reuters) - Trump campaign adviser ...  December 30, 2017   
4  SEATTLE/WASHINGTON (Reuters) - President Donal...  December 29, 2017

In [6]:
# Print the column details for True News DataFrame
true_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21417 entries, 0 to 21416
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   21417 non-null  object
 1   text    21417 non-null  object
 2   date    21417 non-null  object
dtypes: object(3)
memory usage: 502.1+ KB


In [7]:
# Label data and combine
true_df['news_label'] = 1
fake_df['news_label'] = 0
df = pd.concat([true_df, fake_df], ignore_index=True)
df.shape

(42834, 4)

In [8]:
# Drop duplicates and check missing
print(df.isnull().sum())
df.drop_duplicates(inplace=True)
df.shape

title         0
text          0
date          0
news_label    0
dtype: int64


(42400, 4)

In [15]:
# Clean text

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import re

# Download NLTK resources if not already present
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')

try:
    nltk.data.find('corpora/wordnet')
except LookupError:
    nltk.download('wordnet')

try:
    nltk.data.find('tokenizers/punkt_tab')
except LookupError:
    nltk.download('punkt_tab')


stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = text.lower()
    text = re.sub(r"\W+", " ", text)
    tokens = nltk.word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words and word not in string.punctuation]
    return " ".join(tokens)

df['clean_text'] = (df['title'] + " " + df['text']).apply(clean_text)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [17]:
# Tokenize for Word2Vec
df['tokens'] = df['clean_text'].apply(lambda x: x.split())

In [18]:
# Train Word2Vec
w2v_model = Word2Vec(sentences=df['tokens'], vector_size=100, window=5, min_count=2, workers=4, sg=1)
w2v_model.train(df['tokens'], total_examples=len(df['tokens']), epochs=10)

(98485543, 103026180)

In [19]:
# Convert to document vectors
def get_doc_vector(tokens):
    vectors = [w2v_model.wv[word] for word in tokens if word in w2v_model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(w2v_model.vector_size)

df['doc_vector'] = df['tokens'].apply(get_doc_vector)
X = np.vstack(df['doc_vector'].values)
y = df['news_label'].values

In [20]:
# Split and train models
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train, y_train)

rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)

svm_model = SVC(probability=True)
svm_model.fit(X_train, y_train)

SVC(probability=True)

In [21]:
# Evaluate models
print("Logistic Regression:")
print(classification_report(y_test, lr_model.predict(X_test)))
print("ROC-AUC:", roc_auc_score(y_test, lr_model.predict_proba(X_test)[:,1]))

print("\nRandom Forest:")
print(classification_report(y_test, rf_model.predict(X_test)))
print("ROC-AUC:", roc_auc_score(y_test, rf_model.predict_proba(X_test)[:,1]))

print("\nSVM:")
print(classification_report(y_test, svm_model.predict(X_test)))
print("ROC-AUC:", roc_auc_score(y_test, svm_model.predict_proba(X_test)[:,1]))

Logistic Regression:
              precision    recall  f1-score   support

           0       0.47      0.50      0.48      4233
           1       0.47      0.45      0.46      4247

    accuracy                           0.47      8480
   macro avg       0.47      0.47      0.47      8480
weighted avg       0.47      0.47      0.47      8480

ROC-AUC: 0.4561939498878351

Random Forest:
              precision    recall  f1-score   support

           0       0.10      0.10      0.10      4233
           1       0.11      0.11      0.11      4247

    accuracy                           0.10      8480
   macro avg       0.10      0.10      0.10      8480
weighted avg       0.10      0.10      0.10      8480

ROC-AUC: 0.02135888809326698

SVM:
              precision    recall  f1-score   support

           0       0.45      0.51      0.48      4233
           1       0.44      0.38      0.40      4247

    accuracy                           0.44      8480
   macro avg       0.44     

In [22]:
# Interpret top words
coeffs = lr_model.coef_[0]
word_importance = {word: coeffs[w2v_model.wv.key_to_index[word]]
                   for word in w2v_model.wv.key_to_index
                   if word in w2v_model.wv and w2v_model.wv.key_to_index[word] < len(coeffs)}
top_words = sorted(word_importance.items(), key=lambda x: abs(x[1]), reverse=True)[:20]
print("Top contributing words to classification:")
for word, weight in top_words:
    print(f"{word}: {weight:.4f}")

Top contributing words to classification:
russian: 0.1442
told: 0.1422
million: 0.1312
year: -0.1231
government: -0.1204
take: -0.1179
say: 0.1159
new: -0.1071
senate: -0.1055
country: 0.1019
right: 0.1016
people: -0.1016
federal: -0.0999
obama: -0.0986
day: -0.0959
bill: -0.0956
comment: 0.0924
could: 0.0912
committee: 0.0867
said: 0.0861


In [23]:
import nltk

# Download NLTK resources if not already present
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')

try:
    nltk.data.find('corpora/wordnet')
except LookupError:
    nltk.download('wordnet')

try:
    nltk.data.find('tokenizers/punkt_tab')
except LookupError:
    nltk.download('punkt_tab')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
